# Práctica 4 - Recurrent Neural Networks y Transformers

### Natalia Martínez García, Lucía Vega Navarrete
### Grupo: AP.11.06

Primero importamos todas las librerías que vamos a necesitar:

In [1]:
import os
import numpy as np
from miditok import REMI, TokenizerConfig

/Users/luciaveganavarrete/anaconda3/envs/AP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Carga y preprocesado del dataset 

Seguimos el ejemplo del enunciado de la práctica para crear el Tokenizer.

In [2]:
# Configurar el tokenizador REMI
config = TokenizerConfig()
tokenizer = REMI(config)

In [3]:
# Definir las rutas de los datos
MIDI_DIR = "midi"
TRAIN_DIR = os.path.join(MIDI_DIR, "train")
TEST_DIR = os.path.join(MIDI_DIR, "test")

# Obtener lista de archivos midi
train_files = sorted([os.path.join(TRAIN_DIR, f) for f in os.listdir(TRAIN_DIR) if f.endswith('.mid')])
test_files = sorted([os.path.join(TEST_DIR, f) for f in os.listdir(TEST_DIR) if f.endswith('.mid')])

In [4]:
def create_sequences(token_ids, tokenizer, seq_length=51):
    """
    Crea secuencias de longitud fija a partir de los tokens de un archivo midi.
    - Si hay menos de 51 tokens: rellena con EOS + PAD
    - Si hay más de 51 tokens: usa ventanas deslizantes
    """
    sequences = [] # Lista para guardar todas las secuencias
    
    # Obtener los ids de los tokens especiales
    # Los vamos a usar cuando el midi no llegue a 51 tokens
    eos_token = tokenizer['EOS_None'] # marca el final de una secuencia
    pad_token = tokenizer['PAD_None'] # token de relleno
    
    # si la canción tiene menos de 51 tokens rellenar
    if len(token_ids) < seq_length:
        # Añadir el token EOS al final para marcar donde termina
        sequence = token_ids + [eos_token]
        # Calcular cuántos tokens PAD necesitamos para llegar a 51
        padding_needed = seq_length - len(sequence)
        # Añadir ese numero de pads
        sequence = sequence + [pad_token] * padding_needed
        sequences.append(sequence)

    # Si tiene 51 o mas tokens usar ventanas deslizantes
    else:
        # extrae 51 tokens desde la primera posición
        # pasa a la segunda y extrae 51 desde esa posición, etc, etc
        for i in range(len(token_ids) - seq_length + 1):
            sequence = token_ids[i:i + seq_length]
            sequences.append(sequence)
    
    return sequences

Esta función toma los tokens de un archivo MIDI y los organiza en ventanas de 51 tokens:
- **Si el archivo tiene menos de 51 tokens**: lo rellenamos con tokens especiales (EOS + PAD)
- **Si el archivo tiene más de 51 tokens**: creamos ventanas deslizantes. La ventana se ma moviendo token a token: primero cogemos los tokens del 1 al 51, luego movemos la ventana una posición y cogemos del 2 al 52, después del 3 al 53, y así sucesivamente hasta llegar al final.

In [5]:
def process_dataset(file_list, tokenizer, seq_length=51):
    """
    Procesa todos los archivos MIDI y crea las secuencias
    """
    all_sequences = [] # Lista para guardar secuencias de todos los archivos
    
    for file_path in file_list:
        # Tokenizar el archivo
        tokens = tokenizer(file_path)
        # Obtener los ids del primer canal (índice 0)
        token_ids = tokens[0].ids
        # Crear secuencias de longitud fija con la funcion de arriba
        sequences = create_sequences(token_ids, tokenizer, seq_length)
        # Añadir todas las secuencias de este archivo a la lista total
        all_sequences.extend(sequences)
    # Convertir a numpy
    return np.array(all_sequences)

Esta función procesa todos los archivos MIDI de una carpeta (train o test). Primero lee cada archivo MIDI, lo tokeniza y extrae solo el primer canal. Luego crea las ventanas de 51 tokens para cada archivo y guarda todas las secuencias juntas.

In [6]:
def prepare_data_for_training(sequences):
    """
    Para una ventana de 51 tokens:
    - Entrada: los primeros 50 tokens
    - Etiqueta: los últimos 50 tokens
    """
    X = sequences[:, :50] # Primeros 50 tokens (del 0 al 49)
    y = sequences[:, 1:] # Últimos 50 tokens (del token 1 al 50)
    
    return X, y

Esta función separa cada ventana de 51 tokens en entrada y salida, ya que vamos a hacer redes que predigan el siguiente token:
- **Entrada (X)**: los primeros 50 tokens (posiciones 0 a 49). Lo que la red recibe como contexto
- **Etiqueta (y)**: los últimos 50 tokens (posiciones 1 a 50). Lo que la red debe predecir


Ahora procesamos de esta manera los conjuntos de entrenamiento y test:

In [7]:
# Procesar el conjunto de entrenamiento
# tokenizar cada archivo y crea ventanas de 51 tokens
train_sequences = process_dataset(train_files, tokenizer, seq_length=51)
# Separar en entrada y salida con ventanas
X_train, y_train = prepare_data_for_training(train_sequences)

print(f"Secuencias de entrenamiento creadas: {len(train_sequences)}")
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de y_train: {y_train.shape}")

# Procesar el conjunto de test
# tokenizar cada archivo y crea ventanas de 51 tokens
test_sequences = process_dataset(test_files, tokenizer, seq_length=51)
# Separar en entrada y salida con ventanas
X_test, y_test = prepare_data_for_training(test_sequences)

print(f"Secuencias de test creadas: {len(test_sequences)}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_test: {y_test.shape}")

Secuencias de entrenamiento creadas: 2894912
Forma de X_train: (2894912, 50)
Forma de y_train: (2894912, 50)
Secuencias de test creadas: 139315
Forma de X_test: (139315, 50)
Forma de y_test: (139315, 50)
